In [19]:
import os
import googlemaps
from datetime import datetime, date
import responses
import numpy as np

gmaps_key = os.environ["GMAPS_API_KEY"]
gmaps = googlemaps.Client(key=gmaps_key)

At the beginning we should approximate the amount of time people statistically spend in chosen category of places.
We also should determine approximate hours appropriate to visit such a place, but later we will also verify opening hours.
General assumptions about each category of sites.

- CATEGORY : TIMESLOT : AVERAGE TIME
- accommodation - we can suggest some places (taking into consideration budget, distance to the city centre and ratings)
- car_services: IGNORE (but car mode with suggesting car plot can be considered)
- christianity: IGNORE - can be suggested in other section
- culture: 9 - 19 : 2h
- family_entertainment: 8 - 20 : 3h
- food_and_drink: 7 - 24: 2h
- hinduism: IGNORE - can be suggested in other section
- islam: IGNORE - can be suggested in other section
- judaism: IGNORE - can be suggested in other section
- nightlife: 21 - 6 : 2h
- shopping: 9 - 20: 2h - shopping mall to be considered
- sports and fitness: 8 - 22 : 3h
- travel services: IGNORE
- welness: 8 - 22 - 2h


In [ ]:
import json
cat_json = {}
with open("..\categories.json") as f:
    cat_json = json.load(f)

class Category:
    def __init__(self, name:str, categories_list:list, ignore:bool, start_time:int = None, end_time:int = None, duration:int = None):
        self.name = name
        self.ignore = ignore
        self.start_time = start_time
        self.end_time = end_time
        self.duration = duration
        self.categories_list = categories_list
        self.probability: int = None
    
    def set_probability(self, prob):
        self.probability = prob

accommodation = Category("accommodation", ["lodging", "rv_park", "campground"], True)
car_services = Category("car services", cat_json["car services"], True)
christianity = Category("christianity", cat_json["christianity"], True)
culture = Category("culture", cat_json["culture"], False, 9, 19, 2)
family_entertainment = Category("family_entertainment", cat_json["family_entertainment"], False, 8, 20, 3)
food_and_drink = Category("food and drink", cat_json["food and drink"], False, 7, 24, 2)
hinduism = Category("hinduism", cat_json["hinduism"], True)
islam = Category("islam", cat_json["islam"], True)
judaism = Category("judaism", cat_json["judaism"], True)
nightlife = Category("nightlife", cat_json["nightlife"], False, 21, 6, 2)
shopping = Category("shopping", cat_json["shopping"], True)
sports_and_fitness = Category("sports and fitness", cat_json["sports and fitness"], False, 8, 22, 3)
travel_services = Category("travel_services", cat_json["travel_services"], True)
wellness = Category("wellness", cat_json["wellness"], False, 8, 22, 2)

cheap_travel = cat_json["cheap_transport"]
expensive_travel = cat_json["expensive_transport"]

categories_list = [accommodation, car_services, christianity, culture, family_entertainment, family_entertainment, food_and_drink, hinduism, islam, judaism, nightlife, shopping, sports_and_fitness, travel_services, wellness]

In [33]:
from enum import Enum
class TravelWith(Enum):
    alone = 1
    with_partner = 2
    with_family = 3

user_preferences = {'nightlife': 0.13586715459823606, 'christianity': 0.2641089591715071, 'judaism': 0.23523988525072737, 'hinduism': 0.12414621661106745, 'islam': 0.0, 'car_services': 0.17760687271753947, 'culture': 0.31209163031092396, 'food_and_drink': 0.23561079038514032, 'sports_and_fitness': 0.06975500285625458, 'wellness': 0.233438245455424, 'family_entertainment': 0.2411984403928121, 'shopping': 0.0, 'travel_services': 0.21448883761962256}
for i in categories_list:
    i.set_probability(user_preferences[i])

user_destination = 'Cracow'
user_start_datatime = datetime(2023,6, 1, 13)
user_end_datatime = datetime(2023,6, 4, 17)
public_transport_accept = True
bicycle_travel_accept = False
car_travel_accept = True
cost_rate = 6 #scale from 0 to 10
travel_with = TravelWith.alone
#By default we assumed that user doesn't mind public transport

class User:
    def __init__(self, preferences:dict, destination:str, start_time:datetime, end_time:datetime, bicycle_travel_accept:bool, car_travel_accept:bool):
        self.preferences = preferences
        self.destination = destination
        self.start_time = start_time
        self.end_time = end_time
        self.bicycle_travel_accept = bicycle_travel_accept
        self.car_travel_accept = car_travel_accept
        self.available_days = []
        self.prepare_available_days()

    def prepare_available_days(self):
        days_in_city = self.end_time.day - self.start_time.day + 1
        print(list(self.preferences.values()))
        prob_values_mean =  np.mean(list(self.preferences.values()))
        prob_values_median =  np.median(list(self.preferences.values()))

        #Here we define time slots for city exploration taking into account user nightlife's preference
        if self.preferences["nightlife"] > 1.5 * prob_values_mean and self.preferences["nightlife"]>=prob_values_median:
            start_hour = 9
            end_hour = 3
        elif self.preferences["nightlife"] <= 1.5 * prob_values_mean and self.preferences["nightlife"] >= prob_values_mean or self.preferences["nightlife"] > 1.5 * prob_values_mean and self.preferences["nightlife"]<prob_values_median:
            start_hour = 8
            end_hour = 24
        else:
            start_hour = 7
            end_hour = 23
        
        for i in range (days_in_city):
            if i == 0:
                if self.start_time.hour > start_hour:
                    curr_day = CityDay(self.start_time.hour, end_hour)
                else: 
                    curr_day = CityDay(start_hour, end_hour)
            elif i == days_in_city - 1:
                if self.end_time.hour > end_hour:
                    curr_day = CityDay(start_hour, end_hour)
                else: 
                    curr_day = CityDay(start_hour, self.end_time.hour)
            else:
                curr_day = CityDay(start_hour, end_hour)
                    
            self.available_days.append(curr_day)
        
    #Function to get n the most interesting categories for user
    def get_the_most_interesting_categories(self):
        #Get 4 the most interesting categories which are not ignored
        return 0    
    
    
        
        
class CityDay:
    def __init__(self, start_hour:int, end_hour:int):
        self.start_hour = start_hour
        self.end_hour = end_hour
        self.places_list = {}
    

user = User(user_preferences, user_destination, user_start_datatime, user_end_datatime, bicycle_travel_accept, car_travel_accept)

[0.51, 0.44, 0.39, 0.36, 0.4, 0.33, 0.44, 0.42, 0.48, 0.29, 0.36]
13


In [ ]:
#ALGORITHM

Input:
- categories_probs: a dictionary with keys as place categories and values as probabilities of interest and visiting sequence
- start_date: start date of tour
- end_date: end date of tour
- transportation_mode: preferred mode of transportation (walking, bicycle, car)

Output:
- A list of tours, one for each day of the tour period, with each tour containing a list of recommended places to visit

1. Calculate the total number of places to recommend based on the length of the tour and the preferred transportation mode
   1.1. Define a maximum distance to travel between places based on the transportation mode
   1.2  Define the maximum number of hours a user is willing to spend traveling in a day
   1.3. Calculate the maximum number of places that can be visited based on the tour length and the maximum distance and time needed to spend in this places
   1.4. Multiply the maximum number of places by the probabilities of interest for each category to determine how many places to recommend from each category

2. Group the recommended places based on their proximity to each other (taking into consideration geographic coordinates)
   2.1. Calculate the distance between each pair of places
   2.2. Group places that are within the maximum distance of each other into clusters
   2.3. For each cluster, recommend the place with the highest rating as the primary destination and the others as alternative options

3. For each day of the tour, recommend a tour based on the recommended places and their opening hours
   3.1. Sort the recommended places by their rating and opening hours
   3.2. Starting from the earliest opening hour, select the next available place that has not been visited yet and add it to the tour
   3.3. If there are no more places available, end the tour for the day

4. Return the list of tours, each containing the recommended places for each day